# A simple univarite example

In [1]:
using JuMP, DynamicPolynomials, MomentTools

In [2]:
X = @polyvar x
p = x^4-1

x⁴ - 1

We minimize nothing under the constraint p1=0, that is, we search the real roots of p. 

In [14]:
using MosekTools; optimizer = optimizer_with_attributes(Mosek.Optimizer, "QUIET" => true)
v, M = minimize(x^2, [p], [], X, 4,optimizer)

(0.9999999997641433, 
A MOMent program with:
A JuMP Model
Minimization problem with:
Variables: 9
Objective function type: GenericAffExpr{Float64,VariableRef}
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 6 constraints
`Array{GenericAffExpr{Float64,VariableRef},1}`-in-`MathOptInterface.PositiveSemidefiniteConeSquare`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Mosek
Names registered in the model: basis, degree, index, moments, monomials, nu, variables, w, y)

We use Mosek optimizer to solve the underlying SDP optimization problem. The first argument $x^2$ of the function `minimize` is the objective function. The second argument is the array of equality constraints, the third is the array of non-negativity constraints (here empty).

The function `minimize` returns `v` the optimal value found (here it is $0$ since we optimize "$0$") and `M` the Moment Model built for the optimization (which type is `MomentTools.MOM.Model`). 

The minimizers can be obtained from `M` as follows:

In [15]:
getminimizers(M)

1×2 Array{Float64,2}:
 -1.0  1.0

The minimizers are presented as a $n\times r$ matrix, where $n$ is the number of variables and $r$ is the number of points. 

An approximation  of the associate measure can be obtained as follows:

In [16]:
getmeasure(M)

([0.4999999999410355, 0.49999999994103583], [-1.0000000000000009 0.9999999999999996])

It corresponds to the measur $0.5\, \delta_{-1} + 0.5\, \delta_{1}$ where $\delta_x$ is the Dirac measure supported on $x$.


Next, we search the root(s) with $x \geq 0$. For that purpose, we minimize `nothing` (i.e. $0$) and add the non-negativity constraint $x\geq 0$:

In [10]:
v, M = minimize(nothing, [p], [x], X, 4,optimizer)
getminimizers(M)

1×1 Array{Float64,2}:
 -1.0000000000000004

Now we search the negative root with maximal value of $x$.

In [13]:
v, M = maximize(x, [p], [-x], X, 4,optimizer)
getminimizers(M)

1×1 Array{Float64,2}:
 -1.0000000000000004